# Homework 1
### Natural Language Processing and Information Extraction,  2020 WS


_This assignment was created by [Judit Acs](http://juditacs.github.io/) for [this course](https://github.com/bmeaut/python_nlp_2018_spring) and is reused with her permission._

## Instructions

This assignment involves implementing a simple part-of-speech tagger in Python, using the Viterbi algorithm.
The exercises should be solved in the order presented in this notebook.

Most exercises include tests which will pass if your solution is correct, but successful test do not guarantee that your solution is correct and some additional tests will be performed on your code after submission.

Test cells cannot be modified and empty cells cannot be deleted. Your solution should replace placeholder lines such as:

    ### YOUR CODE HERE
    raise NotImplementedError()

You don't need to write separate functions except when the function header is predefined.
Variable names must be derived from the public test.
Please do not add new cells, they will be ignored by the autograder.

### IMPORTANT
Before submitting your solution,
run your notebook with `Kernel -> Restart & Run All` and make sure it
runs without exceptions.
**If your code fails the public tests (the ones you see), you will automatically receive 0 points for that exercise.**

## Submission

Please see the separate set of instructions on TUWEL on how to submit your solution. Note that you can submit multiple times before the deadline, but only your last submission will be graded.

In [1]:

import numpy as np
from collections import defaultdict
import os
import types

# Data setup

You will work with a sample of appr. 1 million words from the UMBC WebBase corpus.
There is also a smaller toy sample of a 100 sentences used for the public test and quick debugging.
Both files are prepackaged as a zip file in your starter repository.

If you are running the code from somewhere else and the data is not there, it will be downloaded and unpacked.
You can set the data directory by changing the value of the `HW2_DATA_DIR` environmental variable.

If you just cloned your starter repository in the recommended way, you should see the data extracted on the first run and no output on later runs.

In [2]:
# setting up data directory (used by the instructor)

data_dir = os.getenv("HW2_DATA_DIR")
if data_dir is None:
    data_dir = ""
    
data_zip_path = os.path.join(data_dir, "data.zip")

if not os.path.exists(data_zip_path):
    print("Download data")
    import urllib.request
    u = urllib.request.URLopener()
    u.retrieve("http://avalon.aut.bme.hu/~judit/resources/umbc/data.zip", data_zip_path)
    print("Data downloaded")

unzip_path = os.path.join(data_dir, "umbc_sample_toy.txt")

if not os.path.exists(os.path.join(data_dir, "umbc_sample_toy.txt")) or \
        not os.path.exists(os.path.join(data_dir, "umbc_sample_1M.txt")):
    print("Extracting data")
    from zipfile import ZipFile
    with ZipFile(data_zip_path) as myzip:
        myzip.extractall(data_dir)
    print("Data extraction done")

# 1. Write a generator function that reads a text corpus. (10 points)

Each sentence occupies two lines. The first line is the sentence itself. Tokens (words) are separated by spaces.
The second line is the POS (part-of-speech) tag of each token, also separated by space.
Then comes an empty line before the next sentence.
The end of the file may or may not contain an empty line.

```
This is a sentence .
DT VBZ DT NN .

This is the next sentence .
DT VBZ DT JJ NN .
```

Your task is to create a generator function with one parameter, the file's name.
The generator should yield one sentence at a time.

A sentence is a list of token-POS tag pairs (tuples), so the first sentence in the example would look like this:

```
[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sentence', 'NN'), ('.', '.')]
```

and the second sentence:

```
[('This', 'DT'), ('is', 'VBZ'), ('the', 'DT'), ('next', 'JJ'), ('sentence', 'NN'), ('.', '.')]

```
Some sentences may be malformatted, and the number of tokens and POS tags differ, **skip** these sentences.

In [43]:
import re

def read_corpus(fn):
    with open(fn) as file:
        for line in file:
            line = line.strip()
            
            if not line:
                continue
            
            tokens = re.split(r'(?<!\d) | (?!\d\/\d)', line)
            pos_tags = next(file).strip().split()
            
            if len(tokens) == len(pos_tags):
                yield list(zip(tokens, pos_tags))

In [44]:
toy_data_fn = os.path.join(data_dir, "umbc_sample_toy.txt")

toy_sentences = read_corpus(toy_data_fn)

assert isinstance(toy_sentences, types.GeneratorType)

toy_sentences = list(toy_sentences)

# there are invalid sentences in the data, that you should skip
assert len(toy_sentences) < 100
assert toy_sentences[0] == [('Then', 'RB'),
 ('Jon', 'NNP'),
 ('falls', 'VBZ'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('pretty', 'JJ'),
 ('new', 'JJ'),
 ('vet', 'NN'),
 ('played', 'VBN'),
 ('by', 'IN'),
 ('Jennifer', 'NNP'),
 ('Love', 'NNP'),
 ('Hewitt', 'NNP'),
 ('.', '.')]

# 2. Basic statistics (15 points)

Compute basic statistics on the corpus. Try to iterate over the file only once and compute all of these statistics as you go.

The statistics you are supposed to compute and store in the appropriate variables (see the tests below):

1. number of sentences,
2. number of tokens (words),
3. number of types (unique words),
4. length of the longest sentence,
5. length of the shortest sentence,
6. word frequencies,
7. POS frequencies.

You are **NOT** allowed to use `collections.Counter` but you are free to use `collections.defaultdict`.

Use the variable `umbc_fn` as a parameter of `read_corpus`.

In [45]:
umbc_fn = os.path.join(data_dir, "umbc_sample_1M.txt")

sentence_no = 0
token_no = 0

sentence_maxlen = 0
sentence_minlen = np.inf

unique_tokens = set()

word_freq = defaultdict(int)
pos_freq = defaultdict(int)

for sentence in read_corpus(umbc_fn):
    sentence_no += 1
    
    sentence_maxlen = max(sentence_maxlen, len(sentence))
    sentence_minlen = min(sentence_minlen, len(sentence))
    
    for token, pos_tag in sentence:
        token_no += 1
        
        unique_tokens.add(token)
        
        word_freq[token] += 1
        pos_freq[pos_tag] += 1
    
type_no = len(unique_tokens)

In [46]:
print("Number of sentences: {}".format(sentence_no))
print("Number of tokens (words): {}".format(token_no))
print("Number of types (unique words)): {}".format(type_no))
print("Length of the longest sentence: {}".format(sentence_maxlen))
print("Length of the shortest sentence: {}".format(sentence_minlen))

Number of sentences: 41989
Number of tokens (words): 969128
Number of types (unique words)): 53727
Length of the longest sentence: 131
Length of the shortest sentence: 2


In [47]:
assert type(sentence_no) == int
assert type(token_no) == int
assert type(type_no) == int
assert type(sentence_maxlen) == int
assert type(sentence_minlen) == int
assert sentence_no == 41989

In [48]:
assert type(word_freq) == defaultdict or type(word_freq) == dict
assert type(pos_freq) == defaultdict or type(pos_freq) == dict

## 2.2 What are the 10 most frequent words and 5 most frequent POS tags.

The lists should be in decreasing order.

In [49]:
most_frequent_words = sorted(word_freq, key=word_freq.get, reverse=True)[:10]
most_frequent_pos = sorted(pos_freq, key=pos_freq.get, reverse=True)[:5]

In [50]:
print(most_frequent_words)
print(most_frequent_pos)
assert type(most_frequent_words) == list
assert type(most_frequent_pos) == list
assert len(most_frequent_words) == 10
assert type(most_frequent_words[0]) == str
assert most_frequent_pos[0] == 'NN'

[',', 'the', '.', 'of', 'a', 'and', 'to', 'in', "'s", 'is']
['NN', 'IN', 'DT', 'JJ', 'NNP']


# 3. Hapax legomenon (10 points)

Hapax legomenon or hapax is a word that only appears once in the dataset. Since the distribution of words roughly follows Zipf's distribution, regardless of the size of the corpus, there will always be a large number of hapax or words that only appear once.

Aside from theoretical, there are technical reasons for filtering these words such as the fact that we have limited memory.

Compute the number of hapax words in the corpus using your previous statistics.
Do not read the file again.

In [51]:
hapax_no = sum(freq == 1 for freq in word_freq.values())

In [52]:
print(hapax_no)
assert type(hapax_no) == int
assert 0 < hapax_no < len(word_freq)

23711


## The `_UNK_` symbol

Rare words including hapax words are most often replaced by a common symbol such as `_UNK_` (unknown).
During inference time (when running your model on unseen data), we may encounter words either never seen in the training data or deemed too rare and replaced with `_UNK_`.
The best way to handle these words is to replace them with `_UNK_` since our model is prepared to handle these symbols.

Follow this strategy in your viterbi function.

Tip: you don't actually have to replace these words, you can check if they're in the `word_idx` dictionary. `dict.get` might be useful.

## 3.1 Set of frequent words.

Create a set of words that appear more at least `min_freq` times in the data.
Do not read the file again.

In [53]:
min_freq = 2

top_words = set(word for word, freq in word_freq.items() if freq >= min_freq)

In [54]:
print(len(top_words))
assert type(top_words) == set
assert 0 < len(top_words) < len(word_freq)
assert "dog" in top_words
assert "titillating" not in top_words

30016


# 4. Mapping and counters (25 points)

## 4.1 Word and POS mapping

Create a word and a POS mapping that maps each word to its unique integer id. The word ids should range from 0 to `len(top_words)-1`, while the POS ids should range from 0 to `len(pos_idx)-1`.
You only need to map the non-rare words.

Include the unknown symbol (`_UNK_`) in the word mapping. What should its id be?

Add a `START` symbol to the POS mapping. It will be used in the Viterbi algorithm.
Do not read the file again.

In [55]:
word_idx = {word: i for i, word in enumerate(top_words)}
word_idx['_UNK_'] = len(word_idx)

pos_idx = {pos_tag: i for i, pos_tag in enumerate(pos_freq)}
pos_idx['START'] = len(pos_idx)

In [56]:
assert len(word_idx) == len(top_words) + 1  # _UNK_ included
assert '_UNK_' in word_idx
assert 'START' in pos_idx

## 4.2 Emission and transition counts


Hidden Markov Models were introduced in the lecture:


* Like the Markov model, we take only the $n$ preceding tokens into consideration
* The idea behind the model is very different:
    * We imagine an automaton that is always in a **(hidden) state**
    * In each state, it emits something we can observe
    * The task is to find out which is _the most probable_ state sequence that generates the observations
* In the POS tagging context,
    * The words in the text are the **observed events**
    * The POS tags are the hidden states
    
    
You will build a simple Hidden Markov Model for English POS tagging.
In this case $n$ equals 1, so only the previous token is taken into consideration.

Transition and emission probabilities are derived from the corpus. Compute the emission and transition counts from the corpus. Do not iterate over the data more than once.

You need to build two matrices as follows ($\#$ means _count of_):

* a $|L| \times |V|$ matrix of integers
  $$M(i,j) = \# \ i^\text{th} \text{ pos tag emitting word } j$$
* an $|L|\times |L|$ matrix of integers
  $$N(i,j) = \# \ j^\text{th} \text{ pos after } i^\text{th} \text { pos}$$
  
**CAUTION** Both matrices are defined differently from the ones in the lab exercise.

### How to handle the POS tag of the first word in a sentence?

HMMs need an explicit way of handling starting probabilities which is usually solved with special start state(s).
Build the transition matrix as if there is an artificial `START` state at the beginning of every sentence.
You don't actually have to add a symbol to the sentence.
The start state does not affect the emission probabilities. 

In [57]:
emission_counts = np.zeros((len(pos_idx), len(word_idx)), dtype=int)
transition_counts = np.zeros((len(pos_idx), len(pos_idx)), dtype=int)

for sentence in read_corpus(umbc_fn):
    i = pos_idx['START']
    
    for token, pos_tag in sentence:
        j = pos_idx[pos_tag]
        transition_counts[i, j] += 1
        
        i, j = j, word_idx.get(token, word_idx['_UNK_'])
        emission_counts[i, j] += 1

In [58]:
print(emission_counts.shape)
print(transition_counts.shape)
assert(emission_counts[pos_idx["NN"], word_idx["_UNK_"]] == 4717)

dog_id = word_idx["dog"]
noun_id = pos_idx["NN"]
vbz_id = pos_idx["VBZ"]
noun_dog_count = emission_counts[noun_id, dog_id]
assert noun_dog_count == 64
print("The state NN emitted the word dog {} times".format(emission_counts[noun_id, dog_id]))
print("The state NN followed the state VBZ {} times".format(transition_counts[noun_id, vbz_id]))

assert type(emission_counts) == np.ndarray
assert type(transition_counts) == np.ndarray

(46, 30017)
(46, 46)
The state NN emitted the word dog 64 times
The state NN followed the state VBZ 8113 times


## 4.3 Emission and transition probabilities

Empirical emission and transition probabilities are defined as

* a $|L|\times |V|$ matrix of floats
  $$P_1(i,j) = \frac{\# \ i^\text{th} \text{ pos tag emitting word } j}{\# \ \text{pos tag } i}$$
* an $|L|\times |L|$ matrix of floats
  $$P_2(i,j) = \frac{\# \ j^\text{th} \text{ pos after } i^\text{th} \text { pos}}{\# \ \text{pos tag } i}$$

Create these probability matrices using the transition and emission counts from the previous exercise.

You should see a warning for emission probabilities since one of the rows sums to 0.
It is the row corresponding to the `START` state.
Add an artificial emission of the `_UNK_` symbol from the `START` state to solve this (see the tests below).

Use 64-bit floats.
Do not use log-probabilities.

In [59]:
emission_probs = np.zeros((len(pos_idx), len(word_idx)), dtype=np.float64)
transition_probs = np.zeros((len(pos_idx), len(pos_idx)), dtype=np.float64)

emission_counts[pos_idx['START'], word_idx['_UNK_']] = 1

for pos_tag, i in pos_idx.items():
    total_count = sum(emission_counts[i])
    for j in word_idx.values():
        emission_probs[i, j] = emission_counts[i, j] / total_count
    
    total_count = sum(transition_counts[i])
    for j in pos_idx.values():
        transition_probs[i, j] = transition_counts[i, j] / total_count

In [60]:
assert np.count_nonzero(emission_counts[pos_idx['START']]) != 0
assert type(emission_probs) == np.ndarray
assert type(transition_probs) == np.ndarray
assert emission_counts.shape == emission_probs.shape
assert transition_counts.shape == transition_probs.shape
assert np.allclose(emission_probs.sum(axis=1), 1)
assert np.allclose(transition_probs.sum(axis=1), 1)

# 5. Viterbi algorithm (30 points)

Implement the Viterbi algorithm for $n=1$ (the model only takes into account the previous state).

The `viterbi` function takes five parameters:
1. the sentence as a list of tokens,
2. the transition probability matrix,
3. the emission probability matrix,
4. the word - id mapping, 
5. and the POS tag - id mapping,

and it returns a list of POS tags.
It is the same length as the input sentence and each POS tag corresponds to one token in the input sentence.

Your function should handle out-of-vocabulary words as if they're `_UNK_` symbols.

Remember that the HMM always starts from an artificial `START` stat, then proceeds to the first _real_ state (POS in this case).

In [61]:
def viterbi(sentence, transitions, emissions, word_idx, pos_idx):
    viterbi_probs = np.zeros((len(pos_idx), len(sentence)), dtype=np.float64)
    backpointers = np.zeros((len(pos_idx), len(sentence)), dtype=np.int)
    
    for j, token in enumerate(sentence):
        word_id = word_idx.get(token, word_idx['_UNK_'])
        
        for pos_tag, i in pos_idx.items():
            # initialize matrices if at start of sentence
            if j == 0:
                viterbi_probs[i, j] = transitions[pos_idx['START'], i] * emissions[i, word_id]
                backpointers[i, j] = pos_idx['START']
                continue
            
            probs = viterbi_probs[:, j - 1] * transitions[:, i] * emissions[i, word_id]
            
            viterbi_probs[i, j] = np.max(probs)
            backpointers[i, j] = np.argmax(probs)
    
    # follow backpointers
    curr_pointer = np.argmax(viterbi_probs[:, j])
    
    pos_idx_to_tag = {i: pos_tag for pos_tag, i in pos_idx.items()}
    pos_tags = [pos_idx_to_tag[curr_pointer]]
    
    for j in range(len(sentence) - 1, 0, -1):
        curr_pointer = backpointers[curr_pointer, j]
        pos_tags.append(pos_idx_to_tag[curr_pointer])
    
    return pos_tags[::-1]

In [62]:
def run_viterbi(sentence):
    """Call viterbi with global parameters from previous exercises"""
    if isinstance(sentence, str):
        sentence = sentence.split()
    return viterbi(sentence, transition_probs, emission_probs, word_idx, pos_idx)

tags = run_viterbi("The cat runs .")
print(tags)
assert tags == ['DT', 'NN', 'VBZ', '.']
work1 = run_viterbi("My work here is done .")
work2= run_viterbi("I work here .")
print("Work should receive different POS tags\n"
      "My work here is done: {}\n"
      "I work here: {}".format(work1[1], work2[1]))
print(viterbi(["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "fox", "."],
              transition_probs, emission_probs, word_idx, pos_idx))

assert run_viterbi("Everyone is dancing now .") == ['NN', 'VBZ', 'VBG', 'RB', '.']
assert run_viterbi("I like fish .") == ['PRP', 'IN', 'NN', '.']
assert run_viterbi("He who laughs last , did not get it .") == ['PRP', 'WP', 'VBZ', 'JJ', ',', 'VBD', 'RB', 'VB', 'PRP', '.']
assert run_viterbi("My favorite machine at the gym is the vending machine .") == ['PRP$', 'JJ', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'DT', 'VBG', 'NN', '.']
assert run_viterbi("We have almost finished the assignment .") == ['PRP', 'VBP', 'RB', 'VBN', 'DT', 'NN', '.']

['DT', 'NN', 'VBZ', '.']
Work should receive different POS tags
My work here is done: NN
I work here: VBP
['DT', 'JJ', 'JJ', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.']


## 5.2 Addtional tests (extra 10 points)

Add your own tests that demonstrate the effect of context in POS tagging such as the previous example, where _work_ has a different POS tag in different context.

2 points / example.

In [63]:
# Example 1
drop1 = run_viterbi("She enjoyed every last drop of coffee .")
drop2 = run_viterbi("I often drop my phone .")
print("Drop should receive different POS tags\n"
      "She enjoyed every last drop of coffee: {}\n"
      "I often drop my phone: {}\n".format(drop1[4], drop2[2]))

# Example 2
fly1 = run_viterbi("Go fly a kite .")
fly2 = run_viterbi("The fly is on the wall .")
print("Fly should receive different POS tags\n"
      "Go fly a kite: {}\n"
      "The fly is on the wall: {}\n".format(fly1[1], fly2[1]))

# Example 3
can1 = run_viterbi("I can speak Finnish .")
can2 = run_viterbi("He drank a can of Coke .")
print("Can should receive different POS tags\n"
      "I can speak Finnish: {}\n"
      "He drank a can of Coke: {}\n".format(can1[1], can2[3]))


# Example 4
watch1 = run_viterbi("I often watch TV .")
watch2 = run_viterbi("Nico has a new watch .")
print("Watch should receive different POS tags\n"
      "I often watch TV: {}\n"
      "Nico has a new watch: {}\n".format(watch1[2], watch2[4]))

# Example 5
right1 = run_viterbi("She is right .")
right2 = run_viterbi("Take a right turn .")
print("Right should receive different POS tags\n"
      "She is right: {}\n"
      "Take a right turn: {}\n".format(right1[2], right2[2]))

Drop should receive different POS tags
She enjoyed every last drop of coffee: NN
I often drop my phone: VB

Fly should receive different POS tags
Go fly a kite: VBP
The fly is on the wall: NN

Can should receive different POS tags
I can speak Finnish: MD
He drank a can of Coke: NN

Watch should receive different POS tags
I often watch TV: VB
Nico has a new watch: NN

Right should receive different POS tags
She is right: RB
Take a right turn: JJ



# PEP8, Code cleanness (10 points)

This cell is here for technical reasons, you will receive feedback on your code quality here. You do not need to write anything here.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()